### TS-Imputation Invocation Notebook — Remote Execution with GPU (CUDA 12) - example 

In [ ]:
from stelar.client import Client, TaskSpec


# Instantiate the client with the base URL and personal credentials
c = Client(base_url="https://klms.stelar.gr", username="***", password="***")

In [2]:
proc = c.processes["759d9454-a3c2-4e37-a4ed-54c8ef0f885e"]
print(f"Browse workflow process at {c._base_url}/console/v1/process/{str(proc.id)}")

Browse workflow process at https://klms.stelar.gr/stelar/console/v1/process/759d9454-a3c2-4e37-a4ed-54c8ef0f885e


In [5]:
t = TaskSpec(name=f"TS-Imputation Remote Example with GPU")

# Set the input missing timeseries file
t.i(missing="56ee301f-0d9e-4433-8b0b-0094647c850a", ground_truth="e4d6257d-a0a4-4b04-bf3d-9ae1c8cc234e")

t.d(alias="d0", dset="afab46b4-42d7-4cbe-84be-798a4b77a167")

# Set the destination path for the imputed timeseries
t.o(
    imputed_timeseries={
        "url": f"s3://test-bucket/test_imputation/gpu_test_imputed_timeseries.csv",
        "resource": {
            "name": f"Imputed timeseries",
            "relation": "imputed_timeseries",
        },
        "dataset": "d0",
    }
)

# Set the parameters for the task
p_spec = {
    "time_column": "time",
    "time_format": "%Y-%m-%d",
    "sep": ",",
    "header": 0,
    "hyperparameter_tuning": False,
    "is_multivariate": False,
    "n_trials": 50,
    "tuning_top_k_fewer_missing": 15000,
    "imp_method": "saits",
    "default": True,
    "params": {}
}

t.p(**p_spec)

# Run the task
ts_impute_t = proc.run(t)
print(
    f"Browse the task at {c._base_url}/console/v1/task/{str(proc.id)}/{str(ts_impute_t.id)}"
)


Browse the task at https://klms.stelar.gr/stelar/console/v1/task/759d9454-a3c2-4e37-a4ed-54c8ef0f885e/0f0524cc-0e3b-4bfe-8dbf-10d42e564882


In [6]:
import subprocess

image_name = "img.stelar.gr/stelar/ts-imputation:latest"

ts_id = ts_impute_t.id
ts_signature = ts_impute_t.signature

docker_command = [
    "docker",
    "run",
    "--rm",
    "--gpus",
    "all",
    image_name,
    "Bearer " + c.token,
    c.api_url.rstrip("/api/"),
    str(ts_id),
    ts_signature,
]

try:
    subprocess.run(docker_command, check=True)
    print("Docker command executed successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error occurred while executing Docker command: {e}")

[STELAR INFO] Performing cURL to the KLMS API to fetch input...
[STELAR INFO] Tool input was fetched from the API endpoint V2.

Executing saits


2025-08-05 12:47:45 [INFO]: Have set the random seed as 2025 for numpy and pytorch.
2025-08-05 12:47:45 [INFO]: No given device, using default device: cuda
2025-08-05 12:47:45 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-08-05 12:47:45 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 200,719
2025-08-05 12:47:46 [INFO]: Epoch 001 - training loss (MSE): 6.9702
2025-08-05 12:47:46 [INFO]: Epoch 002 - training loss (MSE): 2.0544
2025-08-05 12:47:46 [INFO]: Epoch 003 - training loss (MSE): 1.7658
2025-08-05 12:47:46 [INFO]: Epoch 004 - training loss (MSE): 1.9822
2025-08-05 12:47:46 [INFO]: Epoch 005 - training loss (MSE): 1.6332
2025-08-05 12:47:46 [INFO]: Epoch 006 - training loss (MSE): 1.5445
2025-08-05 12:47:47 [INFO]: Epoch 007 - training loss (MSE): 1.3550
2025-08-05 12:47:47 [INFO]: Epoch 008 - training loss (MSE): 1.2737
2025-08-05 12:47:47 [INFO]: Epoch 009 - training loss (MSE): 1.1465
2

{
    "message": "TS-Impute Tool Executed Successfully",
    "output": {
        "imputed_timeseries": "s3://test-bucket/test_imputation/gpu_test_imputed_timeseries.csv"
    },
    "metrics": {
        "Imputation results for saits": {
            "Missing value percentage": 3.2709532949456173,
            "Mean absolute error": 0.18026496718565863,
            "Mean square error": 0.05690228417202577,
            "Root mean square error": 0.23854199666311543,
            "Mean relative error": 0.2288612626919132,
            "Euclidean Distance": 9.64842665440507,
            "r2 score": 0.9385171267824609
        }
    },
    "status": "success"
}

[STELAR INFO] Performing cURL to the KLMS API to propagate output...
[STELAR INFO] The output request to the KLMS API returned:
{
  "help": "Hidden for security reasons",
  "result": null,
  "success": true
}

[STELAR INFO] Tool output was propagated to the API endpoint V2.
Docker command executed successfully.
